In [1]:
# 1. Install (run once)
!pip install -q langchain langchain-community langchain-chroma langchain-huggingface pypdf2 openai chromadb langchain-openai pypdf

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 3.3 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 32.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 19.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.4/21.4 MB 85.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 278.2/278.2 kB 26.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 89.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 62.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 473.8/473.8 kB 33.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.3/103.3 kB 9.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.4/17.4 MB 94.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.5/72.5 kB 7.4 MB/s eta 0:0

In [2]:
# CELL 2: Import Libraries
from langchain_community.document_loaders import PyPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_chroma import Chroma
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_openai import ChatOpenAI
import os

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.3/84.3 kB 1.6 MB/s eta 0:00:00


In [3]:
# 3. OpenRouter setup
os.environ["OPENROUTER_API_KEY"] = "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"  # Your key
os.environ["OPENROUTER_BASE_URL"] = "https://openrouter.ai/api/v1"

In [5]:

# 4. Load your local PDF
loader = PyPDFLoader("/content/terms_and_conditions.pdf")   # Change filename
docs = loader.load()
print(f"Loaded {len(docs)} pages")

Loaded 5 pages


In [6]:
# 5. Split
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
splits = text_splitter.split_documents(docs)
print(f"Created {len(splits)} chunks")

Created 20 chunks


In [7]:
# 6. Embeddings + Chroma (auto-saved)
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

vectorstore = Chroma.from_documents(
    documents=splits,
    embedding=embeddings,
    persist_directory="./chroma_db"   # Automatically saved here
)
print("Vector DB ready and saved to ./chroma_db")


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

...

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Vector DB ready and saved to ./chroma_db


In [8]:
# 7. Retriever
retriever = vectorstore.as_retriever(search_kwargs={"k": 4})

# 8. LLM via OpenRouter (FREE model)
llm = ChatOpenAI(
    model="meta-llama/llama-3.3-70b-instruct:free",   # or google/gemini-2.0-flash-lite-preview:free
    base_url="https://openrouter.ai/api/v1",
    api_key=os.environ["OPENROUTER_API_KEY"],
    temperature=0.2
)


In [14]:

# 9. Prompt
template = """Answer the question based only on the following context:

{context}

Question: {question}
Answer:"""
prompt = ChatPromptTemplate.from_template(template)

# 10. Helper
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

# 11. RAG chain
rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)


In [15]:
# 12. Test
print(rag_chain.invoke("What is the return policy?"))

According to section 8, the return policy is as follows:

* Non-defective standard Goods may be returned to Seller only upon written approval of Seller.
* The return must be freight prepaid to a point designated by Seller.
* A minimum restocking charge of 15% of the purchase price will be applied.
* Goods returned without the authorization of Seller will be refused, and all charges incident to such refusal (such as storage, redelivery, return freight, etc.) will be borne by Buyer.
